In [40]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_excel("GrunnvannsBorehull.xlsx", sheet_name="EnergiBrønn",usecols='E,F,G')
df = df.sort_values('boreDato')
datokolonne = 2
pd.to_datetime(df.iloc[:,datokolonne])

35099   1900-01-01
28004   1900-01-01
24489   1900-01-01
5095    1900-01-01
23840   1900-01-01
           ...    
36713          NaT
38215          NaT
39069          NaT
41600          NaT
50360          NaT
Name: boreDato, Length: 54730, dtype: datetime64[ns]

In [42]:
def bronner_hvert_aar_borer(borer,aar):
    #input:     borer:  Datasett med boredata for valgt oppdragstaker, bruk df for alle oppdragstakere
    #           aar:    Året hvor man vil telle opp antall brønner i
    #output:            Datasett med boredata for valgt oppdragstaker og valgt år
    borer = borer.sort_values('boreDato')
    datoer_borer = borer.iloc[:,datokolonne]
    datoer_aar = [date for date in borer.iloc[:,datokolonne] if date.year == aar]
    indeks_aar = []
    for i in range(0,len(datoer_aar)):
        hvor = np.where(datoer_borer == np.datetime64(datoer_aar[i]))
        indeks_aar = np.concatenate((indeks_aar,hvor[0]))
    if len(indeks_aar) >=1:
        bronner_aar_borer = borer.iloc[int(indeks_aar[0]):int(indeks_aar[-1])+1,:]
    else:
        bronner_aar_borer = pd.DataFrame()
    return bronner_aar_borer

alle_2022 = bronner_hvert_aar_borer(df,2022)
alle_2023 = bronner_hvert_aar_borer(df,2023)

In [43]:
unike_2022 = alle_2022.drop_duplicates(subset='oppdragstaker')
unike_2022 = unike_2022.iloc[:,1]
unike_2022 = unike_2022.sort_values()
unike_2022 = unike_2022.reset_index()
unike_2022 = unike_2022.iloc[:,1]

unike_2023 = alle_2023.drop_duplicates(subset='oppdragstaker')
unike_2023 = unike_2023.iloc[:,1]
unike_2023 = unike_2023.sort_values()
unike_2023 = unike_2023.reset_index()
unike_2023 = unike_2023.iloc[:,1]

print(unike_2023)

0                            ABE BORING AS
1                               BORTECH AS
2          BRÅTEN BRØNN OG ENERGIBORING AS
3                            BY OG BYGD AS
4                 Brustugun Brønnboring as
5                       Brødrene Myhre A/S
6        Brønn og Energiboring Buskerud AS
7                         Båsum Boring A.S
8                Båsum Boring Trøndelag AS
9                    Edland Brønnboring as
10                    FOSEN BRØNNBORING AS
11                  Fetveit Brønnboring AS
12                  Fimreite Bergboring AS
13               Fjellheim Brønnboring A/S
14                    Follo brønnboring AS
15                              GEOTECH AS
16                        HITECH ENERGY AS
17                           HYDRODRILL AS
18                  Haugrud Brønnboring AS
19    Hålogaland Brønn- og Energiboring AS
20      Innlandet Brønn og Energiboring as
21                   Jensen Brønnboring AS
22                             KIVIJOKI AS
23         

In [44]:
antall_2022 = []
antall_2023 = []
for i in unike_2023:
    antall_2022.append(len(alle_2022[alle_2022['oppdragstaker'].str.contains(str(i))]))
    antall_2023.append(len(alle_2023[alle_2023['oppdragstaker'].str.contains(str(i))]))
antall_2022 = pd.DataFrame({'Totalt antall brønner i 2022': antall_2022})
antall_2023 = pd.DataFrame({'Totalt antall brønner i 2023': antall_2023})

antall_enkelthusholdning_2023 = []
antall_storre_anlegg_2023 = []
antall_andre_anlegg_2023 = []
for j in unike_2023:
    hver_borer = alle_2023[alle_2023['oppdragstaker'].str.contains(str(j))]
    antall_enkelthusholdning_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('Enkelthusholdning')]))
    antall_storre_anlegg_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('Større anlegg')]))
    antall_andre_anlegg_2023.append(len(hver_borer[hver_borer['brønnBrukOmfang'].str.contains('|'.join(['Gårdsbruk', 'Hytte/fritidsbolig', 'Turistnæring', 'Ukjent']))]))

antall_enkelthusholdning_2023 = pd.DataFrame({'Antall brønner for enkelthusholdning i 2023': antall_enkelthusholdning_2023})
antall_storre_anlegg_2023 = pd.DataFrame({'Antall brønner i større anlegg i 2023': antall_storre_anlegg_2023})
antall_andre_anlegg_2023 = pd.DataFrame({'Antall brønner i andre typer anlegg i 2023': antall_andre_anlegg_2023})

In [45]:
ferdig = pd.concat([unike_2023,antall_2022,antall_2023,antall_enkelthusholdning_2023,antall_storre_anlegg_2023,antall_andre_anlegg_2023],axis=1)
ferdig = ferdig.sort_values('Totalt antall brønner i 2023', ascending=False)
ferdig.to_excel('Aktive_bronnborere.xlsx')